In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import argparse
import onnxruntime as rt
import numpy as np
from inference_trt import init_decoder_inputs
from IPython.display import Audio
import models
from inference import checkpoint_from_distributed, unwrap_distributed, load_and_setup_model, prepare_input_sequence
from tacotron2_common.utils import to_gpu, get_mask_from_lengths

# Load model.onnx form directory
encoder = rt.InferenceSession('model_onnx/encoder_2.onnx') 
decoder = rt.InferenceSession('model_onnx/decoder_iter_2.onnx')
postnet = rt.InferenceSession('model_onnx/postnet_2.onnx')
waveglow = rt.InferenceSession('model_onnx/waveglow.onnx')


text = input("please enter your text: ")

while text:

    ## Encoder 
    input_length = [50]
    sequences, lengths = prepare_input_sequence([text],cpu_run=True)
    sequences =  np.int64(sequences.cpu())
    while len(sequences) < input_length[0]:
        sequences = np.append(sequences,0)
    sequences = np.expand_dims(sequences,0)
    memory, processed_memory, lens = encoder.run(["memory","processed_memory","lens"],{"sequences":sequences,"sequence_lengths":input_length})
    memory, processed_memory, lens = torch.tensor(memory),torch.tensor(processed_memory),torch.tensor(lens)

    ## Decoder_iter
    mel_lengths = np.zeros([memory.size(0)], dtype=np.int32)
    not_finished = np.ones([memory.size(0)], dtype=np.int32)
    mel_outputs, gate_outputs, alignments = (np.zeros(1), np.zeros(1), np.zeros(1))


    (decoder_input, attention_hidden, attention_cell, decoder_hidden,
         decoder_cell, attention_weights, attention_weights_cum,
         attention_context, memory, processed_memory,
         mask) = init_decoder_inputs(memory, processed_memory, torch.tensor(input_length))

    (decoder_input, attention_hidden, attention_cell, decoder_hidden,decoder_cell, attention_weights, attention_weights_cum,attention_context, memory, processed_memory,mask)  = decoder_input.numpy(), attention_hidden.numpy(), attention_cell.numpy(), decoder_hidden.numpy(),decoder_cell.numpy(), attention_weights.numpy(), attention_weights_cum.numpy(),attention_context.numpy(),memory.numpy(), processed_memory.numpy(),mask.numpy()

    gate_threshold = 0.6
    max_decoder_steps = 1000
    first_iter = True

    while True:
        decoder_output_name = ["decoder_output", "gate_prediction","out_attention_hidden", "out_attention_cell","out_decoder_hidden", "out_decoder_cell","out_attention_weights", "out_attention_weights_cum","out_attention_context"]
        decoder_input_element = {"decoder_input":decoder_input, "attention_hidden": attention_hidden, "attention_cell":attention_cell, "decoder_hidden":decoder_hidden ,"decoder_cell":decoder_cell, "attention_weights":attention_weights, "attention_weights_cum":attention_weights_cum,"attention_context":attention_context, "memory":memory, "processed_memory":processed_memory,"mask":mask}

        (mel_output, gate_output,
                     attention_hidden, attention_cell,
                     decoder_hidden, decoder_cell,
                     attention_weights, attention_weights_cum,
                     attention_context) = decoder.run(decoder_output_name , decoder_input_element)

        if first_iter:
            mel_outputs = np.expand_dims(mel_output, 2)
            gate_outputs = np.expand_dims(gate_output, 2)
            alignments = np.expand_dims(attention_weights, 2)
            first_iter = False
        else:
            mel_outputs = np.concatenate((mel_outputs, np.expand_dims(mel_output, 2)), 2)
            gate_outputs = np.concatenate((gate_outputs, np.expand_dims(gate_output, 2)), 2)
            alignments =np.concatenate((alignments, np.expand_dims(attention_weights, 2)), 2)

        dec = torch.le(torch.sigmoid(torch.Tensor(gate_output)), gate_threshold).to(torch.int32).squeeze(1)
        not_finished = not_finished*dec.numpy()
        mel_lengths += not_finished

        if np.sum(not_finished) == 0:
                print("Stopping after ",mel_outputs.size," decoder steps")
                break
        if mel_outputs.size == max_decoder_steps:
            print("Warning! Reached max decoder steps")
            break

        decoder_input = mel_output

    #postnet
    mel_outputs_postnet = postnet.run(['mel_outputs_postnet'],{'mel_outputs':mel_outputs})

    stride = 256 # value from waveglow upsample
    n_group = 8
    z_size2 = (mel_outputs_postnet[0].shape[2]*stride)//n_group
    z = np.random.randn(1, n_group, z_size2)
    audio = waveglow.run(['audio'],{'mel':mel_outputs_postnet[0].astype('float32'),'z':z.astype('float32')})
    audio_numpy = np.squeeze(audio[0])
    audio_numpy.shape
    rate = 22050
    Audio(audio_numpy, rate=rate)
    
    text = input("please enter your text: ")

In [1]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Using cache found in C:\Users\n_int/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
C:\Users\n_int/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
C:\Users\n_int/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (lin

In [2]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Using cache found in C:\Users\n_int/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (1): Conv1d(51

In [3]:
text = "Hello world, I missed you so much."

In [4]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

Using cache found in C:\Users\n_int/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [5]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050